In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random

In [2]:
pattern = os.path.join('tr', "*.txt")
spam_files = glob.glob(pattern)
spam_counts = {}

pattern = os.path.join('ham', "*.txt")
ham_files = glob.glob(pattern)
ham_counts = {}

def get_counts(counts, files, sample_size=256):
    ix = torch.randint(0, len(files), (256,)).tolist()
    for x in ix:
        fp = files[x]
        f = open(fp, 'r')
        lines = f.read().split('\n')[5:]
        for line in lines:
            words = line.split(' ')
            words = [w.replace('\t', '').replace('!', '').replace('.', '').replace(',', '').replace('?', '').replace('*', '').lower() for w in words]
            for w in words:
                if counts.get(w): counts[w] += 1
                else: counts[w] = 1
                    
get_counts(spam_counts, spam_files)
get_counts(ham_counts, ham_files, sample_size=1024)

In [3]:
ss = sum(spam_counts.values())
print(ss)

# WHITELIST
spam_counts[' '] = 1
spam_counts[''] = 1

remove = []
for (k,v) in spam_counts.items():
    if v/ss > 0.02 or v/ss <= 3/ss: remove.append(k)
for r in remove: spam_counts.pop(r)



sorted_spam_counts = sorted(spam_counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_spam_counts[:10])

103258
[('and', 2022), ('to', 1886), ('the', 1815), ('of', 1295), ('\xad', 1051), ('\u2007͏', 1043), ('a', 925), ('you', 913), ('in', 857), ('for', 854)]


In [4]:
sh = sum(ham_counts.values())
print(sh)

remove = []
for (k,v) in ham_counts.items():
    if v/sh > 0.02 or v/sh <= 3/ss: remove.append(k)
for r in remove: ham_counts.pop(r)

sorted_ham_counts = sorted(ham_counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_ham_counts[:10])

153580
[('and', 2515), ('the', 2447), ('to', 1876), ('for', 1432), ('\u200c', 1359), ('of', 1333), ('a', 1307), ('with', 1124), ('off', 1118), ('your', 1051)]


In [5]:
def spam_prob(w):
    ss = sum(spam_counts.values())
    sh = sum(ham_counts.values())
    ps = 0
    if spam_counts.get(w): ps = spam_counts[w]
    ph = 0
    if ham_counts.get(w): ph = ham_counts[w]
    p = (ps / ss) / ((ps / ss) + (ph / sh))
    return p

#for item in sorted_spam_counts:
#    print(f"{item[0]} : {spam_prob(item[0])}")
spam_words = []
for item in sorted_spam_counts:
    if spam_prob(item[0]) > 0.85:
        spam_words.append(item[0])
for w in spam_words: print(w)
print(len(spam_words))

 ͏
͏
university
best
campus
product
promotion
admissions
sponsored
deals
visit
overstock
promo
virtual
jsoegondo@mycanyonsedu
twitter
hall
arizona
colleges
rochester
admission
o
image
pokémon
discount
experience
website
tour
message
>
smith
hobart
william
clearance
seton
club
limitations
games
overstockcom
baylor
banner
site
sale
great
2022
everything
major
world
friday
lmu
she
promotions
2021
florida
open-box
redeem
jsoegondo@gmailcom
faculty
dame
apps:
“o”
web
xbox
notre
tours
small
tvs
come
s
majors
interested
session
it's
confirm
reference
authorized
ny
codes
disclaimer
wronsky
clara
game
south
process
newegg
to:
cell
gundam
model
phones
tulane
sessions
her
niantic
individual
1/144
concordia
memberships
hope
rit
“cars”
counselor
careers
info
ut
falcon
hws
shoes”
sold
flexible
hear
degree
guide
send
computers
alt_text
relevant
director
experiences
orange
marquette
furniture”
restrictions
steps
chat
honors
poetry
orleans
recommendations
appears
overstockca
appearing
expert
neweggcom
